### Code Hist.

 - CODE : KIER Data_전처리 (결측 제거)  
  - DATE  
    &ensp; 2023-10-12 Created  
    &ensp; 2023-11-21 Code 최신화  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 1) 기존 모듈화된 함수 사용  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 2) 기존 KIER 변환 관련 코드를 모두 통합  
    &ensp; 2024-01-19 Code 개선  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 1) 공통코드 사용 경로 개선, Raw로 저장  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 2) HOT (온수 사용량) 전용 Code 작성  
    &ensp; 2024-01-30 Code 개선 후 백업  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 1) 순시사용량 보간 코드 작성 (미완)  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 2) 백업 후 코드 최적화    
    &ensp; 2024-02-05 Code 개선  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 1) 순시사용량 보간 코드 작성 (Method 02 1차 완성)  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; A. 순시값을 보간하는 방법은 3가지로 제안 가능  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; a) 단순 Linear 보간 (앞뒤 평균치)  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; [Not Recommanded] 매우 긴 구간의 결측을 커버하기에 부적절  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; (양 끝이 0인 경우, 해당 구간을 제대로 보간 불가)      
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; b) 직전 주 동일시간 사용량  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 나쁘지 않은 방법이지만, 직전 주의 데이터가 존재하지 않을 경우 보간 불가  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;(데이터 상 첫 주 케이스 등)  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; c-1) 해당 동일년도/동일월/동일요일/동일시간의 평균 사용량  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 나쁘지 않은 방법이지만, 과정이 조금 복잡해짐  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; c-2) 해당 동일 일시의 348 세대의 평균 사용량  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; c-1보다 기대되는 적용 데이터량이 많음.  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 한파/폭염/우천 등 예외적인 날씨에 따른 사용량 변동의 영향이 적어질 것으로 판단.  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; B. 순시값을 보간하기 전 단계  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; a) Resample시, 시간이 NaN인 구간은 그 간격에서 비어있는 행이다.  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 해당 행의 순시값을 NaN으로 처리   
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; b-1) 시점 n의 적산량이 0이고 시점 n + 1의 적산량이 정상 경우  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 해당 행의 순시값을 NaN으로 처리  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; b-2) 시점 n의 순시량이 음수인 경우  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 해당 행의 순시값을 NaN으로 처리  
    &ensp; 2024-02-07 Code 개선  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 1) 결측 처리 부분 정확도 개선  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 2) Raw Data 상 결측치가 15% 이상일 경우, 해당 데이터를 사용하지 않는 부분 추가  
    
 - DESC  
    &ensp; : int_Domain 코드의 변경에 따라,   
    &emsp; KIER 에너지 사용량 데이터를 동/층/세대별로 분리    

    &emsp; 또한, Raw Data과 세대별 데이터 추출 단계에서,  
    &emsp; 아래와 같은 작업 진행  

    &emsp; [수행 작업]  
    &emsp; 1) Date Column에 대한 유효성 검사 및 이상 Data에 대한 소거  
    &emsp; 1-1) Datetime 유효성 확인  
    &emsp; 1-2) Datetime 이상치 제거  
    &emsp; 2) HOUSE_ID_HO의 해시값을 int_house_num으로 변경 (용량 절감 목적)  
 - DATA  
    1) Period :   
    2) Interval :   
 - Link  
    &ensp; : 

## 01. Code

### 01-01. Init

#### 01-01-01. Init_Module Import

#### Module import

In [1]:
#region Basic_Import
## Basic
import os
os.path.dirname(os.path.abspath('__file__'))
import sys
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('__file__'))))

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from pandas import DataFrame, Series

import math
import random

## Datetime
import time
import datetime as dt
from datetime import datetime, date, timedelta

import glob
from glob import glob
import requests
import json

## 시각화
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 8]

from scipy import stats

## TSNE
from sklearn.datasets import load_digits
from sklearn.manifold import TSNE

## 정규화
# from sklearn.preprocessing import MinMaxScaler, StandardScaler
## Modeling, Model Training
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
## Model 평가
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error, mean_squared_error, r2_score   # model.score   

## Excel/CSV
import openpyxl, xlrd

import urllib
from urllib.request import urlopen
from urllib.parse import urlencode, unquote, quote_plus

from selenium import webdriver
from selenium.webdriver.chrome.service import Service

from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

## Init.
pd.options.display.float_format = '{:.10f}'.format
#endregion Basic_Import

In [2]:
## Import_DL
str_tar = "tf"
## For Torch
if str_tar == "torch":
    import torch
    import torch.nn as nn
    from torch.nn.utils import weight_norm
    print("Torch Imported")
## For TF
elif str_tar == "tf":
    import tensorflow as tf
    import tensorflow_addons as tfa
    print("Tensorflow Imported")
else:
    print("Error : Cannot be used except for Keywords")
    print(" : torch / tf")

Tensorflow Imported


In [3]:
## Import_Local
from Src_Dev_Common import DEV_Common_Data_Datetime as com_date
from Src_Dev_Common import DEV_KASI_Holiday as com_Holi
from Src_Dev_Common import DEV_Common_Data_Analysis as com_Analysis
from Src_Dev_Common import DEV_Common_Data_Preprocessing as com_Prep
from Src_Dev_Common import DEV_Common_Data_Visualization as com_Visual
from Src_Dev_Common import DEV_KMA_Weather_ASOS as com_ASOS
from Src_Dev_Common import DEV_KDHC_Usage as com_KDHC
from Src_Dev_Common import DEV_KIER_Usage as com_KIER

### 01-01-02. Config (Directory, Params)

In [4]:
## Init_config
SEED = 42

np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = "1"
random.seed(SEED)

In [5]:
## Define Todate str
str_now_ymd = pd.datetime.now().date()
str_now_y = pd.datetime.now().year
str_now_m = pd.datetime.now().month
str_now_d = pd.datetime.now().day
str_now_hr = pd.datetime.now().hour
str_now_min = pd.datetime.now().minute

print(pd.datetime.now())
print(str(str_now_y) + " / " + str(str_now_m)  + " / " + str(str_now_d))
print(str(str_now_hr) + " : " + str(str_now_min))

2024-02-29 10:31:00.454092
2024 / 2 / 29
10 : 31


In [6]:
## Dict_Domain
dict_domain = {0:"ELEC", 1:"HEAT", 2:"WATER", 3:"HOT_HEAT", 4:"HOT_FLOW", 99:"GAS"} ## GAS는 사용하지 않음.
int_domain = 3
str_domain = str(dict_domain[int_domain])

dict_col_accu = {0 : "ACTUAL_ACCU_EFF" ## ELEC
                 , 1 : "ACCU_HEAT" ## HEAT
                 , 2 : "ACCU_FLOW" ## WATER
                 , 3 : "ACCU_HEAT" ## HOT 열량
                 , 4 : "ACCU_FLOW" ## HOT 유량
                 , 99 : "ACCU_FLOW" ## GAS
                 }
str_col_accu = str(str_domain + "_" + str(dict_col_accu[int_domain]))

dict_col_inst = {0 : "INST_EFF" ## ELEC_ACCU/INST_EFF
                , 1 : "INST_HEAT" ## HEAT_ACCU/INST_HEAT
                , 2 : "INST_FLOW" ## WATER_ACCU/INST_FLOW
                , 3 : "INST_HEAT" ## HOT_ACCU/INST_HEAT
                , 4 : "INST_FLOW" ## HOT_ACCU/INST_FLOW
                , 99 : "INST_FLOW" ## GAS_ACCU/INST_FLOW
                } 
str_col_inst = str(str_domain + "_" + str(dict_col_inst[int_domain]))

## Directory Root
str_dirData = "../data/data_Energy_KIER/"
str_dir_raw = '../data/data_Energy_KIER/KIER_0_Raw/'
str_dirName_bld = '../data/data_Energy_KIER/KIER_1_BLD/'
str_dirName_f = '../data/data_Energy_KIER/KIER_2_F_' + str_domain + '/'
str_dirName_h = '../data/data_Energy_KIER/KIER_3_H_' + str_domain + '/'

## File
str_fileRaw = str('KIER_RAW_' + str_domain + '_2023-11-12.csv')
str_fileRaw_hList = str('KIER_hList_' + str_domain + '.csv')

print(str(os.listdir(str_dirData)) + "\n")
print(os.listdir(str_dirName_h))

['.ipynb_checkpoints', 'BS_CONFIGURATION_202309251452.csv', 'Data_Population_SJ_2014_2018_1st_Interpolated.csv', 'Data_Population_SJ_2014_2018_D_1st_Interpolated.csv', 'Data_Population_SJ_2014_2018_H_1st_Interpolated.csv', 'DATE_1M_2023-10-20.csv', 'ENV_SENSOR_HST_202309251452.csv', 'F1_MAIN_POWER_HST_202309251503.csv', 'F1_SUB_POWER_HST_202309251510.csv', 'HEATPUMP_HST_202309251520.csv', 'KIER_0_Raw', 'KIER_1_BLD', 'KIER_2_F_ELEC', 'KIER_2_F_GAS', 'KIER_2_F_HEAT', 'KIER_2_F_HOT', 'KIER_2_F_WATER', 'KIER_3_H_ELEC', 'KIER_3_H_GAS', 'KIER_3_H_HEAT', 'KIER_3_H_HOT', 'KIER_3_H_HOT_FLOW', 'KIER_3_H_HOT_HEAT', 'KIER_3_H_WATER', 'KIER_ASOS_WEATHER_DAILY_202309251521.csv', 'KIER_ASOS_WEATHER_HOUR_202309251521.csv', 'KIER_List_Table_Column_2023-09-25.xlsx', 'KIER_Query_2023-09-25.txt', 'KMA_ASOS_119_2010_2023_1st_to CSV.csv', 'LIGHT_HST_202309251523.csv', 'MACHINE_ROOM_HEAT_HST_202309251534.csv', 'MACHINE_ROOM_POWER_APT_HST_20221205_202309251617.csv', 'MACHINE_ROOM_POWER_APT_HST_20221205_202309

In [7]:
## "KIER_01-01_Data_hList.ipynb"로부터 만들어진 Bld/F/H List
df_kier_hList = pd.read_csv(str_dir_raw + str_fileRaw_hList
                            , index_col = 0)
print(df_kier_hList.columns)
df_kier_hList

Index(['HOUSE_ID_DONG', 'HOUSE_ID_HO_PRE', 'HOUSE_ID_HO_INT', 'HOUSE_ID_HO'], dtype='object')


,HOUSE_ID_DONG,HOUSE_ID_HO_PRE,HOUSE_ID_HO_INT,HOUSE_ID_HO
0,561,1,1,NkI4M0EzNDYxQTE1MDJERjM4NURERTcyRjdGQTJDMDU=
63302,561,1,2,ODBEOTE0QzZCMjg5RUMzNTRBQjM1NjE0RDNBNDE0NzU=
126605,561,1,3,QTUwRUJDQTlBMkZENUQyNjg0RDRGREVFMTNGNDI3NUY=
189893,561,1,4,RkRCQzY4OTlCMTRDQjgzNUZBREE3Njc2MkM2MUM2QjI=
252954,561,2,1,MTQxNkQ1RDAzOTE5NTM5RUM3MzlDQkY3RTI2NERCQTM=
...,...,...,...,...
21692970,563,23,2,QkJCQjQwOThFNjgwNjUwQzJGRUJFNDlCNTREOEUxNDU=
21756267,563,23,3,QUJENzFFMEFDRjM4MjY3NTRDQzJCRjdDNUQ3ODA0N0Y=
21819571,563,23,4,RUVENzZDNTc4MUY4NDg5MUREQjZEOUM1M0Q2RjYzMDE=
21882854,563,24,1,NEVENDc1OTFFNDBBRjBDQzkzMTlEMDgzMkMzMkZFQTg=


## 01-02. Data Load (df_raw)

### 01-02-01. KIER (Energy Usage)

### 순시사용량

In [8]:
list_dong = df_kier_hList['HOUSE_ID_DONG'].drop_duplicates()

str_file = 'KIER_' + str_domain + '_INST_10MIN.csv'
df_tmp = pd.read_csv(str_dirName_h + str_file
                        , index_col = 0)
df_tmp['METER_DATE'] = pd.to_datetime(df_tmp['METER_DATE'])
df_tmp = df_tmp.reset_index()

df_tmp

,index,METER_DATE,YEAR,MONTH,DAY,HOUR,MINUTE,HOT_HEAT_INST_561-1-1,HOT_HEAT_INST_561-1-2,HOT_HEAT_INST_561-1-3,...,HOT_HEAT_INST_563-22-5,HOT_HEAT_INST_563-22-6,HOT_HEAT_INST_563-23-1,HOT_HEAT_INST_563-23-2,HOT_HEAT_INST_563-23-3,HOT_HEAT_INST_563-23-4,HOT_HEAT_INST_563-24-1,HOT_HEAT_INST_563-24-2,MEAN_OF_INST,SUM_OF_INST
0,0,2022-07-17 23:20:00,2022,7,17,23,20,0.0000000000,0.0000000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000368732,0.0125000000
1,1,2022-07-17 23:30:00,2022,7,17,23,30,0.0000000000,0.0000000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0001000000,0.0000000000,0.0000000000,0.0000000000,0.0000266272,0.0090000000
2,2,2022-07-17 23:40:00,2022,7,17,23,40,0.0000000000,0.0000000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000238095,0.0080000000
3,3,2022-07-17 23:50:00,2022,7,17,23,50,0.0000000000,0.0000000000,0.0000000000,...,0.0001000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000264264,0.0088000000
4,4,2022-07-18 00:00:00,2022,7,18,0,0,0.0000000000,0.0000000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000307463,0.0103000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64739,64739,2023-10-10 13:10:00,2023,10,10,13,10,0.0000000000,0.0000000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0004000000,0.0000000000,0.0000000000,0.0000351585,0.0122000000
64740,64740,2023-10-10 13:20:00,2023,10,10,13,20,0.0000000000,0.0000000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000495652,0.0171000000
64741,64741,2023-10-10 13:30:00,2023,10,10,13,30,0.0000000000,0.0000000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000398844,0.0138000000
64742,64742,2023-10-10 13:40:00,2023,10,10,13,40,0.0000000000,0.0000000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000216138,0.0075000000


In [1]:
for i in range(0, len(df_tmp)) : df_tmp.loc[i] = df_tmp.loc[i].fillna(df_tmp['MEAN_OF_INST'].loc[i])
df_tmp

NameError: name 'df_tmp' is not defined

모든 값이 NaN이어서 평균 순시 사용량이 계산되지 않은 경우 존재 (ELEC 기준 약 1,372건)  
해당 부분은 단순 선형 보간을 진행

In [10]:
df_tmp['MEAN_OF_INST'] = df_tmp['MEAN_OF_INST'].interpolate()

In [11]:
## 사용량 컬럼에 대한 2차 보간 진행 (선형 보간된 평균 순시 사용량으로 보간)
for i in range(0, len(df_tmp)):
    df_tmp.loc[i] = df_tmp.loc[i].fillna(df_tmp['MEAN_OF_INST'].loc[i])
df_tmp

,index,METER_DATE,YEAR,MONTH,DAY,HOUR,MINUTE,HOT_HEAT_INST_561-1-1,HOT_HEAT_INST_561-1-2,HOT_HEAT_INST_561-1-3,...,HOT_HEAT_INST_563-22-5,HOT_HEAT_INST_563-22-6,HOT_HEAT_INST_563-23-1,HOT_HEAT_INST_563-23-2,HOT_HEAT_INST_563-23-3,HOT_HEAT_INST_563-23-4,HOT_HEAT_INST_563-24-1,HOT_HEAT_INST_563-24-2,MEAN_OF_INST,SUM_OF_INST
0,0,2022-07-17 23:20:00,2022,7,17,23,20,0.0000000000,0.0000000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000368732,0.0125000000
1,1,2022-07-17 23:30:00,2022,7,17,23,30,0.0000000000,0.0000000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0001000000,0.0000000000,0.0000000000,0.0000000000,0.0000266272,0.0090000000
2,2,2022-07-17 23:40:00,2022,7,17,23,40,0.0000000000,0.0000000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000238095,0.0080000000
3,3,2022-07-17 23:50:00,2022,7,17,23,50,0.0000000000,0.0000000000,0.0000000000,...,0.0001000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000264264,0.0088000000
4,4,2022-07-18 00:00:00,2022,7,18,0,0,0.0000000000,0.0000000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000307463,0.0103000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64739,64739,2023-10-10 13:10:00,2023,10,10,13,10,0.0000000000,0.0000000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0004000000,0.0000000000,0.0000000000,0.0000351585,0.0122000000
64740,64740,2023-10-10 13:20:00,2023,10,10,13,20,0.0000000000,0.0000000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000495652,0.0171000000
64741,64741,2023-10-10 13:30:00,2023,10,10,13,30,0.0000000000,0.0000000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000398844,0.0138000000
64742,64742,2023-10-10 13:40:00,2023,10,10,13,40,0.0000000000,0.0000000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000216138,0.0075000000


In [12]:
## 합계 순시량 다시 계산 
list_col = df_tmp.columns[7:-2]

df_kier_Calc = df_tmp[list_col]
df_kier_extract = df_tmp
df_kier_extract['SUM_OF_INST'] = df_kier_Calc.sum(axis = 1)

str_file = 'KIER_' + str_domain + '_INST_10MIN_MEANINTERPOLATED.csv'       
df_kier_extract.to_csv(str_dirName_h + str_file)

In [13]:
df_tmp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64744 entries, 0 to 64743
Columns: 357 entries, index to SUM_OF_INST
dtypes: datetime64[ns](1), float64(350), int64(6)
memory usage: 176.3 MB


In [14]:
# pd.set_option("display.max_columns", 999)
# # pd.set_option("display.max_seq_items", None)
# pd.set_option("display.max_rows", None)

# df_tmp.isnull().sum()

In [15]:
df_kier_extract

,index,METER_DATE,YEAR,MONTH,DAY,HOUR,MINUTE,HOT_HEAT_INST_561-1-1,HOT_HEAT_INST_561-1-2,HOT_HEAT_INST_561-1-3,...,HOT_HEAT_INST_563-22-5,HOT_HEAT_INST_563-22-6,HOT_HEAT_INST_563-23-1,HOT_HEAT_INST_563-23-2,HOT_HEAT_INST_563-23-3,HOT_HEAT_INST_563-23-4,HOT_HEAT_INST_563-24-1,HOT_HEAT_INST_563-24-2,MEAN_OF_INST,SUM_OF_INST
0,0,2022-07-17 23:20:00,2022,7,17,23,20,0.0000000000,0.0000000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000368732,0.0128318584
1,1,2022-07-17 23:30:00,2022,7,17,23,30,0.0000000000,0.0000000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0001000000,0.0000000000,0.0000000000,0.0000000000,0.0000266272,0.0092662722
2,2,2022-07-17 23:40:00,2022,7,17,23,40,0.0000000000,0.0000000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000238095,0.0082857143
3,3,2022-07-17 23:50:00,2022,7,17,23,50,0.0000000000,0.0000000000,0.0000000000,...,0.0001000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000264264,0.0091963964
4,4,2022-07-18 00:00:00,2022,7,18,0,0,0.0000000000,0.0000000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000307463,0.0106997015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64739,64739,2023-10-10 13:10:00,2023,10,10,13,10,0.0000000000,0.0000000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0004000000,0.0000000000,0.0000000000,0.0000351585,0.0122351585
64740,64740,2023-10-10 13:20:00,2023,10,10,13,20,0.0000000000,0.0000000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000495652,0.0172486957
64741,64741,2023-10-10 13:30:00,2023,10,10,13,30,0.0000000000,0.0000000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000398844,0.0138797688
64742,64742,2023-10-10 13:40:00,2023,10,10,13,40,0.0000000000,0.0000000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000216138,0.0075216138
